#segmentation할 data의 경로

segmentation을 수행할 데이터의 경로를 data_dir에 입력하세요.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# segementation 수행할 데이터 경로
data_dir = '/content/drive/MyDrive/HUFS/Digital_image_processing/project/brats18_test'

# output file을 저장할 경로 설정
outdir = '/content/drive/MyDrive/HUFS/Digital_image_processing/project/segmentation_result'

import os

if os.path.exists(data_dir):    # 지정된 경로가 존재하는지 여부를 확인
    print("경로 존재")
else:
    print("경로 존재X")

경로 존재


#segmentaion수행

In [ ]:
# set up environment

!pip install monai-weekly
#!pip install pytorch-ignite
#!pip install tmp

import logging
import sys
import shutil
import time
import tmp

import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd
import monai
from monai.config import print_config
from monai.data import CacheDataset, DataLoader, Dataset,ArrayDataset
from monai.transforms import (
    LoadImaged,
    AddChanneld,
    NormalizeIntensityd,
    ToTensord,
    #ResizeWithPadOrCropd,
    #RandFlipd,
    #EnsureChannelFirst,
    #EnsureChannelFirstd,
    #AsDiscrete,
    #SpatialCropd,
    #RandCropByPosNegLabeld,
    #RandSpatialCropd,
    #RandSpatialCropSamplesd,
    #CropForeground,
    #CropForegroundd,
    #RandGaussianNoised,
    #Spacing,
    #Spacingd,
    #AddChannel,
    #Compose,
    #LoadImage,
    #Resize,
    #EnsureType,
    #Randomizable,
    #LoadImaged,
    #EnsureType,
    #EnsureTyped,
    #AddChanneld,
    #RandAffined,
    #Resized,
    #RandShiftIntensityd,
    #ScaleIntensityRanged,
    #ScaleIntensityRange,
    #MapTransform,
    #RandSpatialCropd,
    #CenterSpatialCropd,
    #Orientationd,
    #Invertd,
    #AsDiscreted
)

from monai.data import DataLoader
from monai.networks.nets import UNet
from monai.data import CacheDataset, DataLoader, Dataset
from monai.inferers import sliding_window_inference
from monai.networks.layers import Norm
import glob
import nibabel as nib

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

# test_set 준비

test_images = sorted(
    glob.glob(os.path.join(data_dir, "Brats18_test_*", "t1ce.nii.gz")))

test_files = [
    {"image": image_name}
    for image_name in (test_images)
]

test_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        NormalizeIntensityd(keys=["image"], nonzero=True, channel_wise=True),
        ToTensord(keys=["image", "label"]),
    ]
)

test_ds = CacheDataset(
    data=test_files, transform=test_transforms, cache_rate=1.0, num_workers=4)

test_loader = DataLoader(test_ds, batch_size=1, num_workers=4)

device = torch.device("cuda:0")
model = UNet(
    dimensions=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

# evaluation
model.load_state_dict(torch.load(
    os.path.join(data_dir, "best_metric_model06748.pth")))
model.eval()
with torch.no_grad():
    for i, test_data in enumerate(test_loader):
        roi_size = (64, 64, 32)
        sw_batch_size = 16
        test_outputs = sliding_window_inference(
            test_data["image"].to(device), roi_size, sw_batch_size, model, overlap=0.42
        )
        plt.figure("check", (18, 6))
        plt.subplot(2, 2, 1)
        plt.title(f"image {i}")
        plt.imshow(torch.sum(test_data["image"][0, 0, :, :, :],dim=1), cmap="gray",aspect=1/4)
        plt.subplot(2, 2, 2)
        plt.title(f"output {i}")
        plt.imshow(torch.amax(torch.argmax(test_outputs.cpu(), dim=1)[0, :, :, :],dim=1),aspect=1/4)
        plt.subplot(2, 2, 3)
        plt.title(f"image {i}")
        plt.imshow(torch.sum(test_data["image"][0, 0, :, :, :],dim=0), cmap="gray",aspect=1/4)
        plt.subplot(2, 2, 4)
        plt.title(f"output {i}")
        plt.imshow(torch.amax(torch.argmax(test_outputs.cpu(), dim=0)[0, :, :, :],dim=0),aspect=1/4)
        plt.show()

        # output파일 저장
        h2 = nib.Nifti1Image(torch.argmax(test_outputs.cpu(), dim=1).numpy().squeeze().astype('float32'),test_data['image_meta_dict']['affine'][0].numpy())
        nib.save(h2,os.path.join(outdir,f"{test_files[i].get('image').split('/')[-2]}.nii.gz"))